# Testing Mistral 7B Instruct - Quantized

This notebook assesses how one may use the open-source 7B instruct LLM created by [Mistral AI](https://mistral.ai/). However, the model is quantized in this notebook.

More details on the impetus of thias notebook can be found [here](https://github.com/Overtrained/contextual-qa-chat-app/issues/15).

## Establish connection to `git` repo

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Change directory according to Google Drive directory

In [2]:
%cd "/content/drive/MyDrive/Colab Notebooks/contextual-qa-chat-app"

/content/drive/MyDrive/Colab Notebooks/contextual-qa-chat-app


In [3]:
!git switch 15-basic-usage-of-the-mistral-7b-llm

Already on '15-basic-usage-of-the-mistral-7b-llm'
Your branch is up to date with 'origin/15-basic-usage-of-the-mistral-7b-llm'.


## Establish environment for running `mistral-7b-instruct`

Below are several set up instllation calls to load the model into the workspace as a quantized model.

In [4]:
%%sh

pip install -q -U bitsandbytes
pip install -q -U git+https://github.com/huggingface/transformers.git
pip install -q -U git+https://github.com/huggingface/peft.git
pip install -q -U git+https://github.com/huggingface/accelerate.git
pip install watermark[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.7 MB/s eta 0:00:00


### Import necessary packages and modules

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [11]:
import watermark

%load_ext watermark

%watermark --hostname --machine --gitbranch --gpu

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.120+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Hostname: 103344f2b9c4

Git branch: 15-basic-usage-of-the-mistral-7b-llm

GPU Info: 
  GPU 0: Tesla T4



## Quantization Configuration

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## Load Model into Workspace

In [7]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Test the Loaded Model

In [9]:
device = "cuda:0"

In [10]:
PROMPT= """ ### Instruction: Act travel influencer on social media.
### Question:
Share some examples of the most populated cities in the world and suggest why I should visit some of them and what makes them different.

### Answer:
"""

encodeds = tokenizer(PROMPT, return_tensors="pt", add_special_tokens=True)
model_inputs = encodeds.to(device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1000,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s>  ### Instruction: Act travel influencer on social media.
### Question:
Share some examples of the most populated cities in the world and suggest why I should visit some of them and what makes them different.

### Answer:
As a travel influencer, I am excited to share with you some of the most populated cities in the world and why you should definitely visit them! 

1. Tokyo, Japan - With a population of over 13 million people, Tokyo is the most populous city in the world. It's known for its bustling streets, cutting-edge technology, and delicious food. Whether you're a foodie, a tech enthusiast, or a fashionista, Tokyo has something for everyone. Don't miss the opportunity to visit the iconic Shibuya Crossing, Tsukiji Fish Market, and Tokyo Tower.

2. Delhi, India - Delhi is one of the oldest cities in the world, with a population of over 18 million people. It's a city of contrasts, where historic landmarks like the Red Fort and Humayun's Tomb coexist with modern high-rises and bust

## Conclusions

The output from the model is solid. When quantized in 4-bit, this model easily was downloaded and ran on a T4 using only ~6 GB of the available 15 GB of RAM.

This means that this method should allow for actual development on my local machine with approx 16 GB of RAM (assuming full functilaity in the M1 chip).